In [ ]:
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import Button, Layout, interact, interactive, fixed, interact_manual
import time
def print_elapsed_time(info_string, sec):
  mins = (sec // 60) % 60
  hours = sec // 3600
  sec = sec % 60
  print(info_string + "{0}h {1}m {2}s".format(int(hours),int(mins),int(sec)))
initial_run = True

In [ ]:
start = time.time()
if initial_run :
  clear_output()
  if initial_run :
    !pip install transformers --quiet
    !pip install transformers[sentencepiece] --quiet
    !pip install sentencepiece --quiet
    clear_output()

    from transformers import T5Tokenizer, T5ForConditionalGeneration

    tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")
    model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")
    clear_output()

    initial_run_checkpoint = time.time(); ET = initial_run_checkpoint - start
    print_elapsed_time("Elapsed time: ", ET)

    initial_run = False

In [ ]:
def run_code():
  start = time.time()
  
  ARTICLE_TO_SUMMARIZE = article_widget.value #article_source
 
  min_percentage = (display_n_widget.value/100)
  max_percentage = ((display_n_widget.value/100)+0.2)
  min_summary_length = int((len(ARTICLE_TO_SUMMARIZE.split()))*min_percentage)
  max_summary_length = int((len(ARTICLE_TO_SUMMARIZE.split()))*max_percentage)

  print()
  print("Panjang Artikel: " + str(len(ARTICLE_TO_SUMMARIZE.split())) + " kata")
  print("Input Persentase Ringkasan: " + str(display_n_widget.value) + "%")
  print("Sedang membuat ringkasan. Mohon tunggu...")

  input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt')
  summary_ids = model.generate(input_ids,
              min_length=min_summary_length,
              max_length=max_summary_length,
              num_beams=10,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True,
              no_repeat_ngram_size=2,
              use_cache=True,
              do_sample = True,
              temperature = 0.8,
              top_k = 50,
              top_p = 0.95)

  summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  clear_output()
  summary_widget = widgets.Textarea(value=summary_text,description='Summary:',disabled=False,layout=Layout(width='50%', height='80px'))
  display(summary_widget)

  print()
  print("Hasil Program")
  print("Panjang Artikel: " + str(len(ARTICLE_TO_SUMMARIZE.split())) + " kata")
  print("Panjang Ringkasan: " + str(len(summary_text.split())) + " kata")
  
  checkpoint = time.time(); ET = checkpoint - start
  print_elapsed_time("Total waktu proses: ", ET)

def display_all_widgets(article_widget, display_n_widget):
  row_1_widget_list = widgets.HBox([article_widget, display_n_widget],layout=Layout(height='130px')); display(row_1_widget_list)
  interact_manual(run_code)

In [ ]:
article_widget = widgets.Textarea(value='Masukan Artikel Disini',placeholder='(insert word here)',description='Artikel:',disabled=False,layout=Layout(width='50%', height='120px'))
display_n_widget = widgets.IntSlider(min=5, max=50, step=5, description='Summary %:', value=10)

display_all_widgets(article_widget, display_n_widget)